In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import os

train = pd.read_json("all/train.json")
test = pd.read_json("all/test.json")

train['all_ingredients'] = train['ingredients'].apply(",".join)
test['all_ingredients'] = test['ingredients'].apply(",".join)

# Modeling

In [2]:
# Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(binary=True).fit(train['all_ingredients'].values)

X_train_vectorized = vect.transform(train['all_ingredients'].values)
X_train_vectorized = X_train_vectorized.astype('float')

Result_transformed = vect.transform(test['all_ingredients'].values)
Result_transformed = Result_transformed.astype('float')

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

encoder = LabelEncoder()
y_transformed = encoder.fit_transform(train.cuisine)

train_x, test_x, train_y, test_y = train_test_split(X_train_vectorized, y_transformed , random_state = 0)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(train_x , train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [8]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=24)
neigh.fit(train_x , train_y)
neigh.predict(test_x, test_y)



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=24, p=2,
           weights='uniform')

In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1)
clf.fit(train_x , train_y)
clf.score(test_x, test_y)

c:\users\bohda\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6971037811745776

In [11]:
from sklearn.linear_model import LogisticRegression

clf1 = LogisticRegression(C=10,dual=False)
clf1.fit(train_x , train_y)
clf1.score(test_x, test_y)

c:\users\bohda\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bohda\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.79434835076428

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

vclf=VotingClassifier(estimators=[('clf1',LogisticRegression(C=10,dual=False)),('clf2',SVC(C=100,gamma=1,kernel='rbf',probability=True))],voting='soft',weights=[1,2])
vclf.fit(train_x , train_y)
vclf.score(test_x, test_y)

In [6]:
y_predicted = neigh.predict(Result_transformed)
y_predicted_final = encoder.inverse_transform(y_predicted)

predictions = pd.DataFrame({'cuisine' : y_predicted_final , 'id' : test.id })
predictions = predictions[[ 'id' , 'cuisine']]
predictions.to_csv('submit.csv', index = False)
